In [347]:
#!pip install -U pip setuptools wheel
#!pip install -U spacy
#!python -m spacy download ru_core_news_sm

In [ ]:
import pandas as pd
import json 
import spacy
from spacy.symbols import nsubj, VERB

spacy.prefer_gpu()
nlp = spacy.load("ru_core_news_sm")

In [6]:
df = pd.read_csv('df_no_deadline_X.csv')
df.head()

,id,Task1.task_text,Task1.task_responsibles_people,Task1.task_responsibles_groups,Global_supervisor,Task2.task_text,Task2.task_responsibles_people,Task2.task_responsibles_groups,Task3.task_text,Task3.task_responsibles_people,...,Task6.task_responsibles_groups,Task7.task_text,Task7.task_responsibles_people,Task7.task_responsibles_groups,Task8.task_text,Task8.task_responsibles_people,Task8.task_responsibles_groups,Task9.task_text,Task9.task_responsibles_people,Task9.task_responsibles_groups
0,0,Закупить книги. Срок завершения: 19 апр 15.,Ответственность за исполнение распоряжения воз...,Административно-хозяйственный отдел,['Ответственность за исполнение распоряжения в...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,Провести анализ поставщиков для замены вышедше...,Ответственность за исполнение распоряжения воз...,Служба ИТ,['Ответственность за исполнение распоряжения в...,Выполнить ремонт кабеля сети передачи данных м...,Контроль над выполнением приказа возложить на ...,Служба ИТ,Создать и настроить виртуальную машину в соотв...,Контроль над исполнением распоряжения оставить...,...,0,0,0,0,0,0,0,0,0,0
2,2,Провести рефакторинг программного обеспечения ...,Контроль над исполнением распоряжения оставить...,Отдел разработки систем,['Контроль над исполнением распоряжения остави...,Провести рефакторинг программного обеспечения ...,Ответственным за исполнение настоящего приказа...,Отдел разработки систем,Провести анализ накопленного технического долг...,За контроль над исполнением распоряжения назна...,...,Отдел разработки систем,0,0,0,0,0,0,0,0,0
3,3,Оплатить командировочные расходы.\nОпределить ...,Контроль над выполнением настоящего распоряжен...,Бухгалтерия,['Контроль над выполнением настоящего распоряж...,Провести аудит персональных данных в информаци...,Обеспечение выполнения возложить на специалист...,Бухгалтерия,Перечислить необходимые средства на закупку об...,Контроль над выполнением настоящего распоряжен...,...,0,0,0,0,0,0,0,0,0,0
4,4,Согласовать смету на выполнение работ компание...,Контроль над выполнением настоящего распоряжен...,Бухгалтерия,['Контроль над выполнением настоящего распоряж...,Выделить средства на проведение обучения.\nУст...,Ответственным за исполнение настоящего приказа...,Бухгалтерия,0,0,...,0,0,0,0,0,0,0,0,0,0


In [297]:
with open('merged_data.json', "r") as readFile:
            merge_data = json.load(readFile)

In [298]:
DEPARTMENTS = []
for n in range(10000):
    try:
        for i in range(1,30):
            DEPARTMENTS.append(merge_data[n]['Tasks']['Task{}'.format(i)]['task_responsibles_groups'])
    except KeyError:
        continue
    except IndexError:
        continue


In [299]:
FIO_POSITION = []
for n in range(10000):
    try:
        for i in range(1,20):
            FIO_POSITION.append(merge_data[n]['Tasks']['Task{}'.format(i)]['task_responsibles_people'])
    except KeyError:
        continue
    except IndexError:
        continue

In [300]:
TASK = []
for n in range(10000):
    try:
        for i in range(1,30):
            TASK.append(merge_data[n]['Tasks']['Task{}'.format(i)]['task_text'])
    except KeyError:
        continue
    except IndexError:
        continue

In [301]:
POSITION = []
for n in range(10000):
    try:
        for i in range(1,30):
            POSITION.append(merge_data[n]['Tasks']['Global_supervisor'][i])
    except KeyError:
        continue
    except IndexError:
        continue

In [ ]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])


Ответственность ROOT Ответственность NOUN [исполнение, Ивановича]
за case исполнение NOUN []
исполнение nmod Ответственность NOUN [за, распоряжения]
распоряжения nmod исполнение NOUN [возложить]
возложить nmod распоряжения NOUN [специалиста]
на case специалиста NOUN []
специалиста obl возложить VERB [на, категории, Кима]
третьей amod категории NOUN []
категории nmod специалиста NOUN [третьей, Службы]
Службы nmod категории NOUN [ИТ]
ИТ nmod Службы PROPN []
Кима appos специалиста NOUN [Юрия]
Юрия flat:name Кима PROPN []
Ивановича nmod Ответственность NOUN []


In [346]:
text = FIO_POSITION[:2000]
keywords_resp = []
keywords_resp_min = []
for n in range(len(text)):
    doc = nlp(text[n])
 
    span = doc[doc[3].left_edge.i : doc[3].right_edge.i+1]
    with doc.retokenize() as retokenizer:
        retokenizer.merge(span)
    for token in doc:
        keywords_resp_min.append(token.head)
    keywords_resp.append(set(keywords_resp_min))




In [246]:
import spacy
text = FIO_POSITION[1]
FIO_1 = []
full_list = []
doc = nlp(text)
span = doc[doc[2].left_edge.i : doc[2].right_edge.i+1]
with doc.retokenize() as retokenizer:
    retokenizer.merge(span)
for token in doc:
        FIO_1.append(token.head.text)
        full_list.append(set(FIO_1))
    
full_list[-1]

{'Ивановича',
 'Ответственность',
 'категории',
 'на',
 'распоряжения',
 'специалиста'}